# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from IPython.display import display

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv('Output/city_data.csv')
city_df.head(1)

,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Río Gallegos,-51.6226,-69.2181,66.25,37,75,16.11,AR,1643831111


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [17]:

locations = city_df[['Lat', 'Lon']]
humidity = city_df['Humidity']
fig = gmaps.figure()

hum_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=100)

fig.add_layer(hum_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# ideal_loca_df = city_df[city_df['Humidity'] > 70]
hotel_loc_df = city_df.query('Temp > 70 and Temp < 80 and `Wind Speed` < 10 and Cloudiness == 0')

hotel_loc_df.head(100)


,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Kapaa,22.0752,-159.3190,74.48,71,0,6.91,US,1643831112
190,Comodoro Rivadavia,-45.8667,-67.5000,78.66,27,0,8.05,AR,1643831147
203,Makakilo City,21.3469,-158.0858,75.18,71,0,5.75,US,1643831174
242,Pisco,-13.7000,-76.2167,77.05,61,0,9.22,PE,1643831082
344,Salalah,17.0151,54.0924,73.49,38,0,2.30,OM,1643831217
425,Atar,20.5169,-13.0499,78.22,18,0,9.60,MR,1643831242
443,Ayorou,14.7308,0.9174,73.11,11,0,8.66,NE,1643831247
475,‘Āhuimanu,21.4447,-157.8378,76.78,72,0,1.99,US,1643831258
520,Kailua,21.4022,-157.7394,77.07,72,0,8.05,US,1643831272
536,Kolokani,13.5728,-8.0339,73.96,13,0,9.04,ML,1643831277


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotels = []
locations2 = hotel_loc_df[["Lat", "Lon"]]
locations2
lats = hotel_loc_df['Lat']
lngs = hotel_loc_df['Lon']
lat_lngs = zip(lats, lngs)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
ext_url = (f"&radius=5000&types=hotel&keyword=Hotels&key={g_key}")


for cords in lat_lngs:
    cords = (f"location={cords[0]},{cords[1]}")
    full_url = (f"{base_url}{cords}{ext_url}")
    # print(full_url)
    v_data = requests.get(full_url).json()
    try:
        print(f"{v_data['results'][0]['name']}")
        hotels.append(v_data['results'][0]['name'])
    except:
        hotels.append('None')
        print(f"None")


# full_url

The ISO
Lucania Palazzo Hotel
Hampton Inn & Suites Oahu/Kapolei
Hotel San Jorge Residencial
Salalah Gardens Hotel
Bab Sahara
Hotel Amenokal
None
None
Lycee Famolo Coulibalibali De Kolokani


In [8]:
#  Combine the hotel name into the Dataframe
hotel_loc_df['Hotel Name'] = hotels
hotel_loc_df

C:\Users\stade\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Kapaa,22.0752,-159.3190,74.48,71,0,6.91,US,1643831112,The ISO
190,Comodoro Rivadavia,-45.8667,-67.5000,78.66,27,0,8.05,AR,1643831147,Lucania Palazzo Hotel
203,Makakilo City,21.3469,-158.0858,75.18,71,0,5.75,US,1643831174,Hampton Inn & Suites Oahu/Kapolei
242,Pisco,-13.7000,-76.2167,77.05,61,0,9.22,PE,1643831082,Hotel San Jorge Residencial
344,Salalah,17.0151,54.0924,73.49,38,0,2.30,OM,1643831217,Salalah Gardens Hotel
425,Atar,20.5169,-13.0499,78.22,18,0,9.60,MR,1643831242,Bab Sahara
443,Ayorou,14.7308,0.9174,73.11,11,0,8.66,NE,1643831247,Hotel Amenokal
475,‘Āhuimanu,21.4447,-157.8378,76.78,72,0,1.99,US,1643831258,None
520,Kailua,21.4022,-157.7394,77.07,72,0,8.05,US,1643831272,None
536,Kolokani,13.5728,-8.0339,73.96,13,0,9.04,ML,1643831277,Lycee Famolo Coulibalibali De Kolokani


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_loc_df.iterrows()]
locations = hotel_loc_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# Display figure
heat_layer = gmaps.heatmap_layer(locations2)



In [11]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))